In [ ]:
# RecursiveJsonSplitter
# 이 JSON 분할기는 JSON 데이터를 깊이 우선 탐색(depth-first traversal)하여 더 작은 JSON 청크(chunk)를 생성합니다.
# 이 분할기는 중첩된 JSON 객체를 가능한 한 유지하려고 시도하지만, 
# 청크의 크기를 min_chunk_size와 max_chunk_size 사이로 유지하기 위해 필요한 경우 객체를 분할합니다. 
# 값이 중첩된 JSON이 아니라 매우 큰 문자열인 경우, 해당 문자열은 분할되지 않습니다.
# 청크 크기에 대한 엄격한 제한이 필요한 경우, 이 분할기 이후에 Recursive Text Splitter를 사용하여 해당 청크를 처리하는 것을 고려해 볼 수 있습니다.

# 분할하는 기준
# -텍스트 분할 방식: JSON 값 기준
# -청크 크기 측정 방식: 문자 수 기준

# %pip install -qU langchain-text-splitters

In [7]:
# requests.get() 함수를 사용하여 "https://api.smith.langchain.com/openapi.json" URL에서 JSON 데이터를 가져옵니다.
# 가져온 JSON 데이터는 json() 메서드를 통해 Python 딕셔너리 형태로 변환되어 json_data 변수에 저장됩니다.

import requests

# JSON 데이터를 로드합니다.
json_data = requests.get("https://api.smith.langchain.com/openapi.json").json()

from langchain_text_splitters import RecursiveJsonSplitter

# JSON 데이터를 최대 300 크기의 청크로 분할하는 RecursiveJsonSplitter 객체를 생성합니다.
splitter = RecursiveJsonSplitter(max_chunk_size=300)

# JSON 데이터를 재귀적으로 분할합니다. 작은 JSON 조각에 접근하거나 조작해야 하는 경우에 사용합니다.
json_chunks = splitter.split_json(json_data=json_data)

# JSON 데이터를 기반으로 문서를 생성합니다.
docs = splitter.create_documents(texts=[json_data])

# JSON 데이터를 기반으로 문자열 청크를 생성합니다.
texts = splitter.split_text(json_data=json_data)

# 첫 번째 문자열을 출력합니다.
print(docs[0].page_content)

print("===" * 20)
# 분할된 문자열 청크를 출력합니다.
print(texts[0])


{"openapi": "3.1.0", "info": {"title": "LangSmith", "version": "0.1.0"}, "servers": [{"url": "https://api.smith.langchain.com", "description": "LangSmith API endpoint."}]}
{"openapi": "3.1.0", "info": {"title": "LangSmith", "version": "0.1.0"}, "servers": [{"url": "https://api.smith.langchain.com", "description": "LangSmith API endpoint."}]}


In [8]:
# 청크의 크기를 확인해 봅시다.
print([len(text) for text in texts][:10])

# 더 큰 청크 중 하나를 검토해 보면 리스트 객체가 있는 것을 볼 수 있습니다.
print(f'*type: {type(texts[3])}')
print(texts[3])

[171, 231, 126, 469, 210, 213, 237, 271, 191, 232]
*type: <class 'str'>
{"paths": {"/api/v1/sessions/{session_id}": {"get": {"parameters": [{"name": "session_id", "in": "path", "required": true, "schema": {"type": "string", "format": "uuid", "title": "Session Id"}}, {"name": "include_stats", "in": "query", "required": false, "schema": {"type": "boolean", "default": false, "title": "Include Stats"}}, {"name": "accept", "in": "header", "required": false, "schema": {"anyOf": [{"type": "string"}, {"type": "null"}], "title": "Accept"}}]}}}}


In [5]:
import json

# paths 만 json 파싱해서 얻어냄.
json_data = json.loads(texts[3])
json_data["paths"]

{'/api/v1/sessions/{session_id}': {'get': {'parameters': [{'name': 'session_id',
     'in': 'path',
     'required': True,
     'schema': {'type': 'string', 'format': 'uuid', 'title': 'Session Id'}},
    {'name': 'include_stats',
     'in': 'query',
     'required': False,
     'schema': {'type': 'boolean',
      'default': False,
      'title': 'Include Stats'}},
    {'name': 'accept',
     'in': 'header',
     'required': False,
     'schema': {'anyOf': [{'type': 'string'}, {'type': 'null'}],
      'title': 'Accept'}}]}}}